# Welcome 

Welcome to the script for generating parameter dictionaries for the recording sessions in your experiment folder. Please follow the upcoming steps in this notebook for further instructions. 

## 1) Import the packages required for running the script

Please run the block of code to import the Python packages that are required for running the rest of this script. 

In [ ]:
#Import required packages
import pickle
import os
from utils.load_probe_info import load_probe_info
import ipywidgets
from ipywidgets import Layout, HBox, VBox
from IPython.display import display

## 2) Enter general parameters for the experiment.

In [ ]:
#Creating widgets for the user input on the general parameters for the experiment

##Main path for the data 

mp_html = ipywidgets.HTML(value = "<p><b>Path to the data of the experiment:</b><br />Enter the path to the folder (with no '/' at the end) that is hierarchically right above the folders of the recording sessions</p>")
mp = ipywidgets.Text(value = "", placeholder = "Enter path for data", disabled = False)
display(VBox([mp_html, mp]))

##Sampling rate
sr = ipywidgets.IntText(value = 30000, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b> Sampling rate (Hz): </b>"),sr]))

##File format
ff_html = ipywidgets.HTML(value = "<p><b>File format:</b><br />(dat for .dat, cont for .continuous, rhd for .rhd)</p>")
ff = ipywidgets.Text(value = 'dat', placeholder = 'Enter file format',
             disabled = False)
display(VBox([ff_html,ff]))

##Number of probes

np_html = ipywidgets.HTML(value = "<p><b>Number of probes:</b><br /><b>WARNING:</b>Pipeline currently supports <b>ONLY</b> the multiple probes being <b>IDENTICAL</b> in type and mapping!!! Pipeline has to be updated before using multiple probes of different types!</p>")
np = ipywidgets.IntText(value = 2, disabled = False)
display(VBox([np_html, np]))

##Probe info
pi_html = ipywidgets.HTML(value = "<b>Type of the probe used in the experiment</b>")
pi = ipywidgets.Dropdown(options=['a4x8_5mm_100_200_177', 'a4x4_tet_150_200_1212'], 
                   value = 'a4x8_5mm_100_200_177', disabled = False)
display(VBox([pi_html, pi]))



## 3) Enter parameters related to the LFP analysis

In [ ]:
#Creating widgets for the user input on the parameters related to the evoked LFP analysis

##doLFPAnalysis

dlfp = ipywidgets.Checkbox(value = False, disabled = False)
display(HBox([ipywidgets.HTML(value = "<b>Select if LFP analysis will be done or not</b>"), dlfp]))

##evokedPre

epre = ipywidgets.FloatText(value = 0.025, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter time taken prior to the stimulus trigger (in s)</b>"), epre]))

##evokedPost

epost = ipywidgets.FloatText(value = 0.100, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter time taken post the stimulus trigger (in s)</b>"), epost]))

#low_pass_freq

lp = ipywidgets.FloatText(value = 300, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b> Enter the cutoff frequency of the low pass filter to extract LFP from data (in Hz)"), lp]))

#notch_filt_freq

nf = ipywidgets.FloatText(value = 50, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b> Enter the frequency of the notch filter (in Hz). Enter 0 if you don't want a notch filter"), nf]))

##cutBeginning

cb = ipywidgets.FloatText(value = 5, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the time to be cut from the beginning of the recording (in s)</b>"), cb]))

##cutEnd

ce = ipywidgets.FloatText(value = 5, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the time to be cut from the end of the recording (in s )"), ce]))

# 4) Enter parameters related to spike sorting

If you are intending to do spike-sorting on this data, please set the spike-sorting parameters. Otherwise, set the boolean parameter "do_spike_sorting" to False below. 

In [ ]:
#Creating widgets for the user input on the parameters related to spike sorting

##doSpikeSorting

dss = ipywidgets.Checkbox(value = False, disabled = False)
display(HBox([ipywidgets.HTML(value = "<b>Select if spike sorting will be done or not</b>"), dss]))

##samplesBefore

sb = ipywidgets.FloatText(value = 0.5, disabled = False)
display(VBox([ipywidgets.HTML(value = '<b>Enter the length of waveform to be taken before the threshold crossing (in ms)'), sb]))

##samplesAfter

sa = ipywidgets.FloatText(value = 1.5, disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the length of waveform to be taken after the threshold crossing (in ms)"), sa]))

##lowCutoff

lc = ipywidgets.FloatText(value = 300., disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the lower cutoff frequency for the bandpass filter to be applied on the raw data (in Hz)"), lc]))

##highCutoff

hc = ipywidgets.FloatText(value = 3000., disabled = False)
display(VBox([ipywidgets.HTML(value = "<b>Enter the higher cutoff frequency for the bandpass filter to be applied on the raw data (in Hz)"), hc]))

# 5) Generate the parameters dictionaries

Please run the block of the code in order to generate the parameters dictionary for each recording session (paramsDict.p) based on the input that you have provided above. 

In [ ]:
dirs = os.listdir(mp.value) #Listing all recording sessions in the main path
probe_info = load_probe_info(pi.value) #Loading probe info from probe selection

for folder in (folder for folder in dirs if ((folder != 'log.txt') and (folder != '.DS_Store') and (folder != 'notes.docx') and (folder != 'other'))):
    p = {} #Parameter dictionary (empty)
    
    #Entering the probe info and electrode mapping into the dictionary
    p['shanks'] = probe_info['numShanks']
    p['probes'] = np.value
    p['probe_type'] = probe_info['type']
    p['probe_name'] = probe_info['name']
    if probe_info['type'] == 'tetrode':
        p['nr_of_tetrodes'] = probe_info['numTetrodes']
        p['max_nr_of_tetrodes_per_shank'] = probe_info['numTetrodesPerShank']
        p['nr_of_groups'] = p['nr_of_tetrodes']
        p['nr_of_electrodes_per_group'] = 4
    elif probe_info['type'] == 'linear':
        p['nr_of_electrodes_per_shank'] = probe_info['numTrodesPerShank']
        p['nr_of_electrodes'] = probe_info['numTrodes']
        p['nr_of_groups'] = p['shanks']
        p['nr_of_electrodes_per_group'] = p['nr_of_electrodes_per_shank']
    p['id'] = probe_info['id']
    
    #Entering the path and file format info into the dictionary
    p['mainpath'] = mp.value
    p['path'] = mp.value + '/' + folder
    p['fileformat'] = ff.value
    
    #Getting the list of 'rhd' files in the folder in case of 'rhd' format
    if p['fileformat'] == 'rhd': 
        p['rhd_file'] = os.listdir(p['path'])
        try: #Removing the non-rhd files from the list, if the session was already analyzed
            p['rhd_file'].remove('paramsDict.p')
            p['rhd_file'].remove('probe.dat')
            p['rhd_file'].remove('probe_evoked.pickle')
            p['rhd_file'].remove('probe_evoked.png')
        except ValueError:
            pass
    
    #Entering the general parameters into the dictionary
    p['sample_rate'] = sr.value
    
    #Entering the LFP analysis parameters into the dictionary
    p['LFP_analysis'] = dlfp.value #This has to be specified for running LFP analysis on the EPhys pipeline
    if dlfp.value:
        p['evoked_pre'] = epre.value
        p['evoked_post'] = epost.value
        p['low_pass_freq'] = lp.value
        p['notch_filt_freq'] = nf.value
        p['cut_beginning'] = cb.value
        p['cut_end'] = ce.value
    
    #Entering the spike-sorting related parameters into the dictionary
    p['spikeSorting'] = dss.value
    if dss.value:
        #Converting the waveform time window parameters from miliseconds to sample counts before saving
        p['samples_before'] = int(sb.value * p['sample_rate'] / 1000) 
        p['samples_after'] = int(sa.value * p['sample_rate'] / 1000)
        p['low_cutoff'] = lc.value
        p['high_cutoff'] = hc.value
    
    #Saving the dictionary in the pickle file named paramsDict.p 
    pickle.dump(p, open((mp.value + '/' + folder + '/paramsDict.p'), 'wb'))
    

# Done!

You might want to check the folders for the recording sessions in your experiment folder to see whether the paramsDict.p files are created in the right place.

Notebook written by Baran Yasar in 04/2017. Please contact him in person or e-mail at yasar@biomed.ee.ethz.ch in case of any questions. 